### Floating FX-Strike Digital Quanto: 
$$
c(S, F, T) = F_{T} * Digital(max(S - K, 0))
$$

The payoff could be regarded as a vanilla digital quanto option with a forward rate agreement(with price 0￥), which is used to convert the fx into dom ccy at time T

So that the floating fx-strike digital quanto is free of exchange risk and correlation risk

$$
c(S, F, 0) = E[Digital(c(S, F, T))] = F_{0} * Digital(max(S - K, 0)) = F_{0} * N(d2)
$$

The calculation of d2 is totally under fx asset measure, there's no quanto adjustment to the risk-neutral probability

In [1]:
import math
import numpy as np
from BlackScholes import BlackScholes
from digital_quanto_pricer import *
from Simulator import Simulator

In [2]:
spot = 106.6
strike = 110.
tau = 0.3
q = 0.
rs = 0.04
rf = 0.05
sigma_s = 0.4
sigma_f = 0.08
rho = -0.1
quanto_factor = 4

In [4]:
time_steps = 1500
dt = tau / time_steps
num_paths = 300000

## Sanity Check using Pseudo Random Number
### vanilla option

In [5]:
%%time
# payoffs = spot * np.exp(np.cumsum((rf - q - 0.5 * sigma_s ** 2) * dt + sigma_s * math.sqrt(dt) * np.random.standard_normal((time_steps, num_paths)), axis=0))
payoffs = Simulator.simulate_pseudo(spot, rf, q, sigma_s, dt, num_paths, time_steps)

CPU times: user 15.2 s, sys: 17.9 s, total: 33.1 s
Wall time: 47.5 s


In [6]:
x = [payoff[-1] for payoff in payoffs]
x = [max(0, item - strike) for item in x]
np.mean(x) * np.exp(-rf * tau)

8.535256994136093

In [7]:
opt = BlackScholes(spot, strike, rf, q, sigma_s, tau, BlackScholes.TAG_CALL)

In [8]:
opt.price()

8.520497276304305

In [9]:
opt.n_d2 * math.exp(-rf * tau)

0.42050044479801657

### Floating rate quanto for call option:

In [10]:
FloatingFXDigitalQuantoCall(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.42050044479801657

In [11]:
x = [1 if item > 0 else 0 for item in x]
np.mean(x)

0.42712666666666665

## Sanity Check using Sobol Random Number
### vanilla option

In [12]:
%%time
payoffs = Simulator.simulate_sobol(spot, rf, q, sigma_s, dt, num_paths, time_steps)

CPU times: user 40.2 s, sys: 49.6 s, total: 1min 29s
Wall time: 1min 48s


In [13]:
x = [payoff[-1] for payoff in payoffs]
x = [max(0, item - strike) for item in x]
np.mean(x) * np.exp(-rf * tau)

8.528007608266323

In [14]:
opt = BlackScholes(spot, strike, rf, q, sigma_s, tau, BlackScholes.TAG_CALL)

In [15]:
opt.price()

8.520497276304305

In [16]:
opt.n_d2 * math.exp(-rf * tau)

0.42050044479801657

### Floating Rate Digital Quanto Call:
#### Denominated in fx price (vanilla digital)

In [17]:
FloatingFXDigitalQuantoCall(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.42050044479801657

In [19]:
x = [1 if item > 0 else 0 for item in x]
np.mean(x) * math.exp(-rf * tau)

0.41990396425580545

### Floating Rate Digital Quanto Put:
#### Denominated in fx price (vanilla digital):

In [20]:
FloatingFXDigitalQuantoPut(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.564611494805046

In [21]:
math.exp(-rf * tau) * (1 - np.mean(x))

0.5652079753472572